In [1]:
import torch 
import torch.nn as nn 
from torch.nn import functional as F

In [9]:
block_size = 8 
batch_size = 4 
max_iters =10000
learning_rate = 3e-3
eval_iters = 250
n_embed = 384
n_layer = 4

In [4]:
with open('wizard_of_oz.txt' , 'r' , encoding='utf-8') as f :
    text = f.read()
chars = sorted(set(text))
vocal_size = len(chars)

In [5]:
string_to_int = {c:i for i,c in enumerate(chars)}
int_to_string = {i:c for i,c in enumerate(chars)}
encode = lambda s : [string_to_int[c] for c in s] 
decode = lambda x : ''.join([int_to_string[i] for i in x])
data = torch.tensor(encode(text) , dtype=torch.long)

In [6]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]
def get_batch(split):
    if split == 'train':
        data = train_data
    else:
        data = val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

x,y = get_batch('train')

In [7]:
eval_iters = 1000
@torch.no_grad()
def estimate_loss():
    out ={}
    model.eval()
    for split in ['train' , 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X,Y)
            losses[k]=loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [ ]:
class gptlanguagemodel(nn.Module):
    def __init__(self , vocal_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocal_size , n_embed)
        self.position_embedding_table = nn.Embedding(block_size , n_embed)
        self.blocks = nn.Sequential(*[Block(n_embed , n_head=n_head ) for _ in range(n_layer)])
        
        self.ln_f = nn.LayerNorm(n_embed)
        self.lm_head = nn.Linear(n_embed , vocal_size)
        